In [18]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [19]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate, NChaoGate
from chaogatenn.maps import (
    LogisticMap,
    DuffingMap,
    LorenzMap,
    RosslerMap,
    ChenMap,
    RosslerHyperchaosMap,
)
from chaogatenn.utils import grad_norm

In [20]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
NOR_Y = jnp.array([1, 0, 0, 0], dtype=bool)  # NOR
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = NAND_Y

In [21]:
# 3 input combinations

X3 = jnp.array(
    [
        [0, 0, 0],
        [0, 0, 1],
        [0, 1, 0],
        [0, 1, 1],
        [1, 0, 0],
        [1, 0, 1],
        [1, 1, 0],
        [1, 1, 1],
    ],
    dtype=bool,
)

AND_Y3 = jnp.array([0, 0, 0, 0, 0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y3 = jnp.array([0, 1, 1, 1, 1, 1, 1, 1], dtype=bool)  # OR gate output

X = X3
Y = AND_Y3

In [22]:
Map = LogisticMap(a=4.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=1.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )
# Map = ChenMap(dt=1e-5, steps=100)
# Map = RosslerHyperchaosMap()

In [23]:
Map(2)  # type: ignore

-8.0

In [24]:
DELTA, X0, X_THRESHOLD = jax.random.normal(jax.random.PRNGKey(42), (3,))
chao_gate = NChaoGate(DELTA=DELTA, X0=X0, X_THRESHOLD=X_THRESHOLD, Map=Map)

In [25]:
[chao_gate(x) for x in X]

[Array(4.0121733e-05, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.00469383, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.00469383, dtype=float32),
 Array(0.00469383, dtype=float32),
 Array(0.03252234, dtype=float32)]

In [26]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch n"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return -jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15))

In [27]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim n"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [28]:
optim = optax.adabelief(3e-5)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [29]:
epochs = 5_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

    if loss < 1e-3:
        break

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0, Loss: 0.4301854074001312, Grad Norm: 3.68650484085083
Epoch 10, Loss: 0.42841774225234985, Grad Norm: 3.6793086528778076
Epoch 20, Loss: 0.42592015862464905, Grad Norm: 3.6692135334014893
Epoch 30, Loss: 0.4227776825428009, Grad Norm: 3.656507730484009
Epoch 40, Loss: 0.41909000277519226, Grad Norm: 3.641512155532837
Epoch 50, Loss: 0.41493698954582214, Grad Norm: 3.6244614124298096
Epoch 60, Loss: 0.41037997603416443, Grad Norm: 3.6055221557617188
Epoch 70, Loss: 0.40546712279319763, Grad Norm: 3.584808588027954
Epoch 80, Loss: 0.4002384841442108, Grad Norm: 3.5624053478240967
Epoch 90, Loss: 0.39472898840904236, Grad Norm: 3.5383787155151367
Epoch 100, Loss: 0.38896897435188293, Grad Norm: 3.5127711296081543
Epoch 110, Loss: 0.3829861879348755, Grad Norm: 3.4856181144714355
Epoch 120, Loss: 0.3768061101436615, Grad Norm: 3.4569482803344727
Epoch 130, Loss: 0.37045279145240784, Grad Norm: 3.4267807006835938
Epoch 140, Loss: 0.3639487624168396, Grad Norm: 3.3951327800750732
Ep

In [30]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: 0.5097967982292175, X0: -1.7298341989517212, X_THRESHOLD: -2.1229114532470703


In [31]:
# Test the trained model by verifying the output of the AND gate
print("\nTest the trained model:")
print("Input | Output")
for x, y in zip(X, Y):
    print(
        f"{x} | {chao_gate.Map(chao_gate.X0 + (x * chao_gate.DELTA ).sum()) > chao_gate.X_THRESHOLD}"
    )



Test the trained model:
Input | Output
[False False False] | False
[False False  True] | False
[False  True False] | False
[False  True  True] | False
[ True False False] | False
[ True False  True] | False
[ True  True False] | False
[ True  True  True] | True


In [32]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=1.0
